# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 1-1
## Учимся переводить между decision/evaluation/search на примере Bin Packing

**Даны:** 
* $[w_1, \dots , w_k], w_i \in [0, 1]$ — веса грузов  
* $n_{\text{bins}}$ — количество корзин грузоподъемностью 1.

Предполагается, что функция `solve_bp_decision(weights, n_bins)` решает «распознавательный» (decision) вариант задачи bin packing. На вход ей подаётся список весов и число корзин. Функция возвращает булевский ответ на вопрос «можно ли заданные веса раскидать по не более чем `n_bins` контейнерам? 

Напишите содержимое функции `solve_bp_evaluation`, которая возвращает оптимальное число корзин (решает evaluation variant задачи bin packing). 

Затем напишите содержимое функции `solve_bp_search`, которая возвращает список номеров корзин, которые при каком-нибудь оптимальном распределении присваиваются весам из списка `weights` (корзины нумеруются с единицы). 

Каждая из следующих функций должна вызывать предыдущую не более чем полиномиальное число раз.

In [ ]:
def solve_bp_decision(weights: list, n_bins: int) -> bool:
    def able_to_pack(weights: list, bin_capacities) -> bool:
        return weights == [] or any( 
            able_to_pack( weights[:-1], bin_capacities[:i] + [capacity - weights[-1]] + bin_capacities[(i + 1):] ) 
            for i, capacity in enumerate(bin_capacities) if capacity >= weights[-1] 
        )

    return able_to_pack( sorted(weights), [1.0] * n_bins )

# To test this, run:
# solve_bp_decision([0.8, 0.09, 0.4, 0.7], 2)
# solve_bp_decision([0.8, 0.09, 0.4, 0.7], 3)

In [ ]:
def solve_bp_evaluation(weights: list) -> int:
    # Вырожденный случай.
    if len(weights) == 0:
        return 0
    
    left = 0
    # Нам точно хватит столько же корзин, сколько весов --
    # просто положим каждый элемент в отдельную корзину.
    right = len(weights)
    # В бинпоиске по допустимому числу корзин сохраняется инвариант:
    # solve_bp_decision(weights, left) = False,
    # solve_bp_decision(weights, right) = True.
    # Число вызовов solve_bp_decision -- O(log(len(weights))).
    while left < right - 1:
        median = (left + right) // 2
        if solve_bp_decision(weights, median):
            right = median
        else:
            left = median
    min_n_bins = right
    return min_n_bins

In [ ]:
def solve_bp_search(weights: list):
    weights_mutable = weights
    n_weights = len(weights)
    n_bins_optimal = solve_bp_evaluation(weights)
    
    solution = [0] * n_weights
    # Для всех весов до обладающего данным номером уже определены корзины.
    not_solved = 0
    # Первая корзина, которую заполнили не полностью.
    not_full = 1
    
    # На каждом нетривиальном шаге цикла мы заполняем корзину:
    # вес, который еще не был куда-то положен, кладется в эту корзину
    # (должен же он где-то лежать), после чего мы смотрим, можем ли мы
    # добавить еще один вес с неопределенной корзиной к нему,
    # не нарушая оптимальности. Если да, то мы можем считать,
    # что объединение весов является одним весом;
    # в weights_mutable пара элементов заменяется одним,
    # а на место второго элемента ставится 0, который
    # не влияет на решение.
    # Число вызовов solve_bp_evaluation не превышает n * (n - 1) / 2.
    for not_solved in range(n_weights):
        # Случай, когда для веса была уже определена корзина.
        if solution[not_solved] != 0:
            continue
        
        solution[not_solved] = not_full
        for i in range(not_solved + 1, n_weights):
            if solution[i] != 0:
                continue
            
            # Вес, который мы добавим в корзину к изучаемому.
            add_weight = weights_mutable[i]
            if add_weight + weights_mutable[not_solved] > 1:
                continue
            
            weights_mutable[not_solved] += add_weight
            weights_mutable[i] = 0
            
            if solve_bp_evaluation(weights_mutable) == n_bins_optimal:
                solution[i] = not_full
            else:
                weights_mutable[not_solved] -= add_weight
                weights_mutable[i] = add_weight
        
        not_full += 1
    
        
    return solution # list